In [0]:
#!hostname -I


In [0]:
# !wget https://www.dropbox.com/s/tce4mdqz8n5xqvs/text_file.txt   # test_file.txt file of 4.00 MB that will be sent
# !wget https://www.dropbox.com/s/g8oq6mm2j1zhkv2/test_file.txt
# !ls


In [1]:
#!rm -r text_file.txt.1
!ls

datalab


In [0]:
import socket, pickle
receiverIP=socket.gethostbyname(socket.gethostname())
from socket import *
senderPort = 12000
receiverPort = 5200 

In [0]:
class Packet:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  data=None,
  type_ =1
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.data=data
    self.type_=type_

class ACK:
  def __init__(
  self,
  seq_num=None,
  checksum=None,
  real=None
  ):
    self.seq_num=seq_num
    self.checksum=checksum
    self.real=real
  

In [0]:
import random

def decision(probability):
    return random.random() < probability

### STOP AND WAIT

In [8]:
import os
import time
t0 = time.time()
time.sleep(1)
rang = 8

def recv_child():
    stats_={}
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))
    print("--The receiver is ready to receive--\n")
    
    actual_seq_num=0
    expected_seq_num=0
    
    while (True):
        (message, senderAddress) = receiverSocket.recvfrom(2048)
        message=pickle.loads(message)
        t6=time.time()
        if expected_seq_num== message.seq_num:
            if decision(0.5):
                receiverSocket.sendto(pickle.dumps(ACK(expected_seq_num, 0,actual_seq_num)), senderAddress)
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - t0)] = 'receiver sent ack for packet '+str(expected_seq_num)+' actual: '+ str(actual_seq_num)
                actual_seq_num += 1
                expected_seq_num = actual_seq_num % 2
                                      
                if message.type_ == 0:
                    time.sleep(0.1)
                    print ('\n')
                    for (key, msg) in stats_.items():
                        print (key, msg)
            else:
                stats_['{0:.6f}'.format(t6 - t0)] = 'receiver dropped ack for packet '+str(expected_seq_num)+' actual: '+str(actual_seq_num)
                actual_seq_num += 1
                expected_seq_num = actual_seq_num % 2
        else:
            #duplicate
            receiverSocket.sendto(pickle.dumps(ACK(message.seq_num, 0, actual_seq_num-1)), (senderAddress))
             
            stats_['{0:.6f}'.format(t6 - t0)] = '--duplicate-- already received ' + str(message.seq_num) + ' actual: ' + str(actual_seq_num-1)
                                  

            
            

    # os._exit(0)
    

def send_parent():
      newpid = os.fork()
      if newpid == 0:
         recv_child()
      else:
         stats_={}
         time.sleep(3)
         pids = (os.getpid(), newpid)
         packets=[]
         for i in range(0,rang):
            
            if (i != rang-1):
                packets.append(Packet(i%2,0,b'kareem'))
            else:
                packets.append(Packet(i%2,0,b'kareem',type_=0))
                
                           
         #print("send_parent: %d, recv_child: %d\n" % pids)
         senderSocket = socket(AF_INET, SOCK_DGRAM)
         senderSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
         senderSocket.bind(('', senderPort))
         message = packets

         
         for i in range(0,rang):
           if decision(0.5):
            senderSocket.sendto(pickle.dumps(message[i]),(receiverIP, receiverPort))
            t1 = time.time()
            stats_["{0:.6f}".format(t1-t0)]="sent packet "+str(i%2)+" actual: "+str(i)
           else:
            t1 = time.time()
            stats_["{0:.6f}".format(t1-t0)]="sent packet "+str(i%2)+" actual: "+str(i)

           while (True):
               
               modifiedMessage=pot_ack=None
               try:
                   senderSocket.settimeout(0.03)
                   modifiedMessage, receiverAddress = senderSocket.recvfrom(2048)
                   t5 = time.time()
                   pot_ack=pickle.loads(modifiedMessage)
                   if isinstance(pot_ack,ACK):
                    if pot_ack.seq_num == i%2:
                     stats_["{0:.6f}".format(t5-t0)]= "sender received ack for packet "+str(pot_ack.seq_num)+" actual: "+str(pot_ack.real)
                     modifiedMessage=None
                    
                     if(i==(rang-1)):
                       for key, msg in stats_.items():
                         print(key,msg)
                      
                     break
               except:
                   senderSocket.settimeout(None)
                   t8 = time.time()
                   stats_["{0:.6f}".format(t8-t0)]="--timeout-- no ack received for packet "+str(i%2)+" actual: "+str(i)
                  
                   if decision(0.5):
                    senderSocket.sendto(pickle.dumps(message[i]),(receiverIP, receiverPort))
                    t7 = time.time()
                    stats_["{0:.6f}".format(t7-t0)]="--again-- sent packet "+str(i%2)+" actual: "+str(i)
                   else:
                    t7 = time.time()
                    stats_["{0:.6f}".format(t7-t0)]="--again-- dropped packet "+str(i%2)+" actual: "+str(i)
                    
         senderSocket.close()

send_parent()

--The receiver is ready to receive--

4.008556 sent packet 0 actual: 0
4.009062 sender received ack for packet 0 actual: 0
4.009403 sent packet 1 actual: 1
4.009657 sender received ack for packet 1 actual: 1
4.009681 sent packet 0 actual: 2
4.039889 --timeout-- no ack received for packet 0 actual: 2
4.039940 --again-- dropped packet 0 actual: 2
4.070145 --timeout-- no ack received for packet 0 actual: 2
4.070377 --again-- sent packet 0 actual: 2
4.100574 --timeout-- no ack received for packet 0 actual: 2
4.100752 --again-- sent packet 0 actual: 2
4.100936 sender received ack for packet 0 actual: 2
4.100988 sent packet 1 actual: 3
4.131175 --timeout-- no ack received for packet 1 actual: 3
4.131377 --again-- sent packet 1 actual: 3
4.131572 sender received ack for packet 1 actual: 3
4.131624 sent packet 0 actual: 4
4.131686 sender received ack for packet 0 actual: 4
4.131717 sent packet 1 actual: 5
4.161919 --timeout-- no ack received for packet 1 actual: 5
4.161965 --again-- dropped pa

In [6]:
'''
def recv_child():
    stats_ = {}
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))
    print("--The receiver is ready to receive--\n")
    actual_seq_num = 0
    expected_seq_num = 0
    while 1:
        
        (message, senderAddress) = receiverSocket.recvfrom(2048)
        t3 = time.time()
        modifiedMessage = pickle.loads(message)
        if not isinstance(modifiedMessage, Packet):
            continue

        seq_no = modifiedMessage.seq_num
        if seq_no != expected_seq_num:
            receiverSocket.sendto(pickle.dumps(ACK(seq_no, 0,
                                  actual_seq_num)), senderAddress)
        
            
            stats_['{0:.6f}'.format(t3 - t0)] = \
                '--duplicate-- already received ' + str(seq_no) \
                + ' actual: ' + str(actual_seq_num)
            continue
            
        else:
            stats_['{0:.6f}'.format(t3 - t0)] = 'received packet ' \
                + str(seq_no) + ' actual: ' + str(actual_seq_num)
            if decision(0.5):
                receiverSocket.sendto(pickle.dumps(ACK(expected_seq_num, 0,actual_seq_num)), senderAddress)
                t6 = time.time()
                #actual_seq_num += 1
                #expected_seq_num = actual_seq_num % 2
                stats_['{0:.6f}'.format(t6 - t0)] = 'receiver sent ack for packet '+str(expected_seq_num)+' actual: '+ str(actual_seq_num)
            
            else:
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - t0)] = 'receiver dropped ack for packet '+str(expected_seq_num)+' actual: '+str(actual_seq_num)
                
            actual_seq_num += 1
            expected_seq_num = actual_seq_num % 2

            if modifiedMessage.type_ == 0:
                time.sleep(0.1)
                print ('\n')
                for (key, msg) in stats_.items():
                    print (key, msg)

'''

'\ndef recv_child():\n    stats_ = {}\n    receiverSocket = socket(AF_INET, SOCK_DGRAM)\n    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)\n    receiverSocket.bind((\'\', receiverPort))\n    print("--The receiver is ready to receive--\n")\n    actual_seq_num = 0\n    expected_seq_num = 0\n    while 1:\n        \n        (message, senderAddress) = receiverSocket.recvfrom(2048)\n        t3 = time.time()\n        modifiedMessage = pickle.loads(message)\n        if not isinstance(modifiedMessage, Packet):\n            continue\n\n        seq_no = modifiedMessage.seq_num\n        if seq_no != expected_seq_num:\n            receiverSocket.sendto(pickle.dumps(ACK(seq_no, 0,\n                                  actual_seq_num)), senderAddress)\n        \n            \n            stats_[\'{0:.6f}\'.format(t3 - t0)] =                 \'--duplicate-- already received \' + str(seq_no)                 + \' actual: \' + str(actual_seq_num)\n            continue\n            \n        else:\n

In [7]:
def recv_child():
    stats_={}
    receiverSocket = socket(AF_INET, SOCK_DGRAM)
    receiverSocket.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
    receiverSocket.bind(('', receiverPort))
    print("--The receiver is ready to receive--\n")
    
    actual_seq_num=0
    expected_seq_num=0
    
    while (True):
        (message, senderAddress) = receiverSocket.recvfrom(2048)
        message=pickle.loads(message)
        t6=time.time()
        if expected_seq_num== message.seq_num:
            if decision(0.5):
                receiverSocket.sendto(pickle.dumps(ACK(expected_seq_num,checksum=0,actual_seq_num), senderAddress)
                t6 = time.time()
                stats_['{0:.6f}'.format(t6 - t0)] = 'receiver sent ack for packet '+str(expected_seq_num)+' actual: '+ str(actual_seq_num)
                actual_seq_num += 1
                expected_seq_num = actual_seq_num % 2
                                      
                if message.type_ == 0:
                time.sleep(0.1)
                print ('\n')
                for (key, msg) in stats_.items():
                    print (key, msg)
            else:
                stats_['{0:.6f}'.format(t6 - t0)] = 'receiver dropped ack for packet '+str(expected_seq_num)+' actual: '+str(actual_seq_num)
                actual_seq_num += 1
                expected_seq_num = actual_seq_num % 2
        else:
            #duplicate
            receiverSocket.sendto(pickle.dumps(ACK(message.seq_num, 0, actual_seq_num-1)), senderAddress)
             
            stats_['{0:.6f}'.format(t6 - t0)] = '--duplicate-- already received ' + str(message.seq_num) + ' actual: ' + str(actual_seq_num-1)
                                  

            
            

SyntaxError: ignored

In [0]:
!ps -fA | grep python

In [0]:
packets=[]
f=open("test_file.txt","rb")
data = f.read(2048)
seqNO=0
while (data):
        packets.append(Packet(seqNO,0,data))
        data=f.read(2048)
        seqNO+=1
print(len(packets))

In [0]:
'''collectedFile=b''
for i in range( len(packets) ):
  collectedFile+=packets[i].data
'''

In [0]:
coFile=open("co_test_file.txt","wb")
for i in range(len(packets)):
  coFile.write(packets[i].data)
coFile.close()
file=open("co_test_file.txt","rb")
new=file.read(2048)
print(new)


In [0]:
from google.colab import files
!ls
files.download('co_test_file.txt')


In [0]:
'''import socket
from socket import *
serverIP = '172.17.0.2'
serverPort = 12000
clientSocket = socket(AF_INET, SOCK_DGRAM)
message = input('Input lowercase sentence:')
clientSocket.sendto(message.encode(),(serverIP, serverPort))
modifiedMessage, serverAddress = clientSocket.recvfrom(2048)
print(modifiedMessage)
clientSocket.close()
'''


In [0]:
'''import socket
from socket import *
serverPort = 12000
serverSocket = socket(AF_INET, SOCK_DGRAM)
serverSocket.bind(('', serverPort))
print("The server is ready to receive")
while 1:
  message, clientAddress = serverSocket.recvfrom(2048)
  modifiedMessage = message.upper()
  serverSocket.sendto(modifiedMessage, clientAddress)
'''